In [1]:
import pandas as pd
pd.set_option("display.precision", 10)
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 300)
#pd.set_option('display.max_colwidth', -1)
from scipy import stats as st
import scipy.stats
import utils as ut
import numpy as np
from tqdm import tqdm
import math

### Data Loading & Settings

In [2]:
event_frame_size = 21 # Counts from the right in excel sheet 21 columns and sets the split between estimation and event window 
event_size = [3,7,11,21] # event window size. This number does not effect estimation window size only event window size


In [3]:
# load data
omxp_index = pd.read_csv("data/OMXPI.xlsx_Sheet1.csv")

In [4]:
# load data
stock_csv = "Financial_Stock_Data"
df_dnb = pd.read_csv ("data/"+stock_csv+".csv", delimiter=",", header=None)
#df_dnb = df_dnb.dropna()
df_dnb

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276
0,Name,12/18/2020,12/21/2020,12/22/2020,12/23/2020,12/24/2020,12/25/2020,12/28/2020,12/29/2020,12/30/2020,12/31/2020,1/1/2021,1/4/2021,1/5/2021,1/6/2021,1/7/2021,1/8/2021,1/11/2021,1/12/2021,1/13/2021,1/14/2021,1/15/2021,1/18/2021,1/19/2021,1/20/2021,1/21/2021,1/22/2021,1/25/2021,1/26/2021,1/27/2021,1/28/2021,1/29/2021,2/1/2021,2/2/2021,2/3/2021,2/4/2021,2/5/2021,2/8/2021,2/9/2021,2/10/2021,2/11/2021,2/12/2021,2/15/2021,2/16/2021,2/17/2021,2/18/2021,2/19/2021,2/22/2021,2/23/2021,2/24/2021,2/25/2021,2/26/2021,3/1/2021,3/2/2021,3/3/2021,3/4/2021,3/5/2021,3/8/2021,3/9/2021,3/10/2021,3/11/2021,3/12/2021,3/15/2021,3/16/2021,3/17/2021,3/18/2021,3/19/2021,3/22/2021,3/23/2021,3/24/2021,3/25/2021,3/26/2021,3/29/2021,3/30/2021,3/31/2021,4/1/2021,4/2/2021,4/5/2021,4/6/2021,4/7/2021,4/8/2021,4/9/2021,4/12/2021,4/13/2021,4/14/2021,4/15/2021,4/16/2021,4/19/2021,4/20/2021,4/21/2021,4/22/2021,4/23/2021,4/26/2021,4/27/2021,4/28/2021,4/29/2021,4/30/2021,5/3/2021,5/4/2021,5/5/2021,5/6/2021,5/7/2021,5/10/2021,5/11/2021,5/12/2021,5/13/2021,5/14/2021,5/17/2021,5/18/2021,5/19/2021,5/20/2021,5/21/2021,5/24/2021,5/25/2021,5/26/2021,5/27/2021,5/28/2021,5/31/2021,6/1/2021,6/2/2021,6/3/2021,6/4/2021,6/7/2021,6/8/2021,6/9/2021,6/10/2021,6/11/2021,6/14/2021,6/15/2021,6/16/2021,6/17/2021,6/18/2021,6/21/2021,6/22/2021,6/23/2021,6/24/2021,6/25/2021,6/28/2021,6/29/2021,6/30/2021,7/1/2021,7/2/2021,7/5/2021,7/6/2021,7/7/2021,7/8/2021,7/9/2021,7/12/2021,7/13/2021,7/14/2021,7/15/2021,7/16/2021,7/19/2021,7/20/2021,7/21/2021,7/22/2021,7/23/2021,7/26/2021,7/27/2021,7/28/2021,7/29/2021,7/30/2021,8/2/2021,8/3/2021,8/4/2021,8/5/2021,8/6/2021,8/9/2021,8/10/2021,8/11/2021,8/12/2021,8/13/2021,8/16/2021,8/17/2021,8/18/2021,8/19/2021,8/20/2021,8/23/2021,8/24/2021,8/25/2021,8/26/2021,8/27/2021,8/30/2021,8/31/2021,9/1/2021,9/2/2021,9/3/2021,9/6/2021,9/7/2021,9/8/2021,9/9/2021,9/10/2021,9/13/2021,9/14/2021,9/15/2021,9/16/2021,9/17/2021,9/20/2021,9/21/2021,9/22/2021,9/23/2021,9/24/2021,9/27/2021,9/28/2021,9/29/2021,9/30/2021,10/1/2021,10/4/2021,10/5/2021,10/6/2021,10/7/2021,10/8/2021,10/11/2021,10/12/2021,10/13/2021,10/14/2021,10/15/2021,10/18/2021,10/19/2021,10/20/2021,10/21/2021,10/22/2021,10/25/2021,10/26/2021,10/27/2021,10/28/2021,10/29/2021,11/1/2021,11/2/2021,11/3/2021,11/4/2021,11/5/2021,11/8/2021,11/9/2021,11/10/2021,11/11/2021,11/12/2021,11/15/2021,11/16/2021,11/17/2021,11/18/2021,11/19/2021,11/22/2021,11/23/2021,11/24/2021,11/25/2021,11/26/2021,11/29/2021,11/30/2021,12/1/2021,12/2/2021,12/3/2021,12/6/2021,12/7/2021,12/8/2021,12/9/2021,12/10/2021,12/13/2021,12/14/2021,12/15/2021,12/16/2021,12/17/2021,12/20/2021,12/21/2021,12/22/2021,12/23/2021,12/24/2021,12/27/2021,12/28/2021,12/29/2021,12/30/2021,12/31/2021,1/3/2022,1/4/2022,1/5/2022,1/6/2022,1/7/2022
1,ADDLIFE B,135.94,133.48,136.78,136.94,136.94,136.94,143.78,140.32,144,144,144,149.46,151.54,151.54,151.92,157.5,151.3,148.62,151.2,157.6,151.6,148.6,148.6,151,151.8,150.6,150.2,151,146.8,147.6,146.2,152.4,161.6,171.8,167,174,166.6,168.8,167

## Stock calculation

In [5]:
# Locate window split in table data 
thres = df_dnb.shape[1]-event_frame_size

# Get event window index values 
lower_idx, upper_idx = ut.event_win_indexes(thres, event_frame_size, event_size)

# Define variables
stocks_data = pd.DataFrame()
stocks_metadata = pd.DataFrame()

# Each data entry takes up 3 rows so loop iterations are divide by 3
for it in tqdm(range(int(df_dnb.shape[0]/3))):

    # Select stock(relevant rows) based on iterator(it) from table 
    df_stock = df_dnb.iloc[0+it*3:2+it*3]
    # convert to list
    stock = df_stock.values.tolist()

    # Check for nan values. If true then skip stock
    if ut.has_nan_values(stock[1], it): continue

    # Convert stock values from strings to floats
    stock[1][1:] = [float(i) for i in stock[1][1:]]
    
    # Get the respective OMXPI values
    OMXPI_values = ut.get_omxpi_values(omxp_index, stock[0])
    stock.append(OMXPI_values)

    # Get the stock return values
    stock_return_values = ut.stock_return_values(stock[1])
    stock.append(stock_return_values)

    # Get the market return values
    market_return_values = ut.market_return_values(stock[2])
    stock.append(market_return_values)

    # Convert newly created data back to dataframe
    df_stock = pd.DataFrame(stock)

    # Define window sizes 
    estimatation_window = ut.get_windows(df_stock, thres, event_size)
    est_win_size = estimatation_window.shape[1]-1

    # Get slope and intercept
    stock_return_estw = estimatation_window.iloc[[3]].values[0][2:].tolist()
    market_return_estw = estimatation_window.iloc[[4]].values[0][2:].tolist()
    intercept, slope, stderr = ut.linear_regression(market_return_estw, stock_return_estw)

    # Compute Standard deviation
    lst1 = df_stock.iloc[3][thres:].to_list()
    lst2 = df_stock.iloc[4][thres:].to_list()
    std_dev = pd.DataFrame(list(zip(lst1, lst2))).std()[0]
    
    # Compute Normal return
    market_return = df_stock.iloc[4].values.tolist()[1:]
    normal_return = ut.normal_return_values(market_return, slope, intercept)
    stock.append(normal_return)
    
    # Compute Abnormal return
    abnormal_return = np.subtract(stock[3][1:],stock[5][1:]).tolist()
    abnormal_return.insert(0,"AbnormalReturn")
    stock.append(abnormal_return)

    # Compute t-statistic R
    abnormal_return = stock[6][1:]
    t_stat_r = ut.t_stat_r(abnormal_return, stderr)
    stock.append(t_stat_r)
    
    # Put the stock data into a table again
    df_stock = pd.DataFrame(stock)
        
    # Get car value
    abnormalReturn_event_window = df_stock.iloc[6][lower_idx:upper_idx]
    car_value = abnormalReturn_event_window.sum()
    

    # Get t-stat CAR
    t_stat_car = car_value / (std_dev*(event_size)**(1/2))
    
    # Collect all stocks together in one table 
    # --------------------------------------------
    # Add stock meta-data to data table 
    row1 = ["df_stockRow", "df_stock#","intercept", "slope", "standard_error", "t_stat_car", "car_value", "eve_win_size"]
    row2 = [[(it*3), (it), intercept, slope, stderr, t_stat_car, car_value, event_size]]
    variables = pd.DataFrame(row2)
    variables.columns = row1
    
    # Collect all stock data 
    stocks_data = pd.concat([stocks_data, df_stock], axis=0)
    stocks_metadata = pd.concat([stocks_metadata, variables], axis=0)
    

  4%|▍         | 15/381 [00:02<01:14,  4.94it/s]

Stock 9 in row: 27 disregarded. nan values present
Stock 10 in row: 30 disregarded. nan values present
Stock 11 in row: 33 disregarded. nan values present
Stock 12 in row: 36 disregarded. nan values present
Stock 13 in row: 39 disregarded. nan values present


 34%|███▍      | 130/381 [00:33<01:04,  3.87it/s]

Stock 130 in row: 390 disregarded. nan values present


 41%|████▏     | 158/381 [00:39<00:50,  4.40it/s]

...

 91%|█████████ | 346/381 [01:29<00:09,  3.63it/s]

Stock 346 in row: 1038 disregarded. nan values present
Stock 347 in row: 1041 disregarded. nan values present
Stock 348 in row: 1044 disregarded. nan values present


 99%|█████████▉| 378/381 [01:37<00:00,  4.03it/s]

Stock 376 in row: 1128 disregarded. nan values present


100%|██████████| 381/381 [01:38<00:00,  3.86it/s]


## Pirate Calculations (part 1)

In [7]:
df_AbnormalReturn = stocks_data.loc[stocks_data[0] == "AbnormalReturn"]

# Means for each column
aar_means = ut.mean(df_AbnormalReturn, lower_idx, upper_idx)

# Standard deviation for each column
aar_stds = ut.st_dev(df_AbnormalReturn, lower_idx, upper_idx)

# 1-sample T-test and P-value
aar_tstats, aar_pvalues = ut.t_test(df_AbnormalReturn, lower_idx, upper_idx)

## Pirate Calculations (part 2)

In [34]:
oddvalues = {}
# AAR Values 
row1 = ["AAR", "standardDev", "Tstat", "Pvalue"]
row2 = [aar_means, aar_stds, aar_tstats, aar_pvalues]
variables = pd.DataFrame(row2)
variables = variables.T
variables.columns = row1

# CAAR Values
ev21 = variables["AAR"].sum()
ev11 = variables["AAR"][5:-5].sum()
ev5 = variables["AAR"][7:-7].sum()
ev3 = variables["AAR"][9:-9].sum()
CAAR = {"ev21": ev21, "ev11": ev11, "ev5": ev5, "ev3": ev3}
oddvalues["caar_value"] = CAAR["ev21"]

# CAAR standard deviations
caar_std = {}
carlist = []
for key, caar in CAAR.items():
    for car in stocks_metadata["car_value"]:
        carlist.append((car-caar)**2)
    caar_std[key] = math.sqrt(sum(carlist)/len(carlist))
oddvalues["caar_std"] = caar_std["ev21"]  

for i in zip(list(CAAR.values()), list(caar_std.values())):  
    ttest = i[0]/i[1]
    oddvalues["caar_pvalue"] = scipy.stats.t.sf(ttest, df=df_dnb.shape[0]/3-1)
    break

# WilCoxon test
oddvalues["willCox_stat"], oddvalues["willCox_pvalue"] = scipy.stats.wilcoxon(stocks_metadata["car_value"])
oddvalues = pd.DataFrame(oddvalues, index=[0])


### SAVE to EXCEL --------------------------------------------------  
# Create a Pandas Excel writer
path = "export/"+stock_csv+"_winsize"+str(event_size)+'.xlsx'
writer = pd.ExcelWriter(path, engine='xlsxwriter')

# Write each dataframe to a different worksheet.
stocks_data.to_excel(writer, sheet_name='data')
stocks_metadata.to_excel(writer, sheet_name='meta-data')
variables.to_excel(writer, sheet_name='AAR')
oddvalues.to_excel(writer, sheet_name='CAAR_n_Cox')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

#### Save data to Excel